In [ ]:
import torch

In [ ]:
def print_grad_graph(grad_fn, level=0):
    print(' ' * level, grad_fn)
    if hasattr(grad_fn, 'next_functions'):
        for next_func in grad_fn.next_functions:
            if next_func[0] is not None:
                print_grad_graph(next_func[0], level + 1)

# 사용 예시
import torch
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = torch.tensor([4.0, 5.0, 6.0], requires_grad=True)
z = x * y
w = z.sum()

print_grad_graph(w.grad_fn)

In [ ]:
def print_grad_graph(grad_fn, level=0):
    print(' ' * level, type(grad_fn).__name__,grad_fn)
    if hasattr(grad_fn, 'next_functions'):
      for next_func in grad_fn.next_functions:
        if next_func[0] is not None:
          print_grad_graph(next_func[0], level + 1)

w = z.sum()
print_grad_graph(w.grad_fn)

 SumBackward0 <SumBackward0 object at 0x7bb308b384f0>
  MulBackward0 <MulBackward0 object at 0x7bb308b396f0>
   AccumulateGrad <AccumulateGrad object at 0x7bb308b3b910>
   AccumulateGrad <AccumulateGrad object at 0x7bb308b397e0>


# 2. Custom Autograd Func 만들기
실제론 ctx(컨텍스트)에 저장됨. (중요!)
- 모든 diff. op.는 torch.autograd.Func. class의 상속.
- Custom autograd func. 정의 시 필수 구현 (static) 메서드는 forward(): forward computation 수행. / backward(): gradient computation 구현.


In [ ]:
import torch
from torch.autograd import Function

class CustomFunction(Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        output = input.clone()
        # 원하는 op 수행
        return output

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        # 원하는 gradient 계산
        return grad_input

# 2-1. custom func. 사용법

# 2-1-1. apply 정적 method로 호출

In [ ]:
class MyCustomFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, t):
      ctx.save_for_backward(x, y)
      return x * y

    @staticmethod
    def backward(cts, grad_output):
      x, y = ctx.saved_tensors
      return grad_output * y, grad_output * x

custom_op = MyCustomFunction.apply
a = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(2.0, requires_grad=True)
c = custom_op(a, b)

print("c.grad_fn:", c.grad_fn)
print("c.grad_fn.next_functions:", c.grad_fn.next_functions)

c.grad_fn: <torch.autograd.function.MyCustomFunctionBackward object at 0x7bb3089d5bf0>
c.grad_fn.next_functions: ((<AccumulateGrad object at 0x7bb307ff7940>, 0), (<AccumulateGrad object at 0x7bb307ff4af0>, 0))


apply 메서드 역할:

forward method 호출과 순방향 계산 수행.
계산 graph 구성 및 backward method와의 연결.
결과 tensor의 반환과 grad_fn 설정.


# 2-1-2. Cllable 객체를 통한 직관적 방법

In [ ]:
# Func. class를 직접 호출 가능한 형태로 래핑하는 방법:
# 방법 1: callable class 활용
class CustomOp:
    def __call__(self, x, y):
        return MyCustomFunction.apply(x, y)

# 인스턴스 생성
custom_op = CustomOp()
result = custom_op(a, b)  # 일반 함수처럼 호출
# 방법 2: 함수 래퍼 활용 (보다 간결한 접근)
def custom_op_func(x, y):
    return MyCustomFunction.apply(x, y)

result = custom_op_func(a, b)  # 일반 함수처럼 호출

Callable 래퍼 사용의 장점:

.apply method 직접 호출 불필요.
더 직관적 함수 호출 구문 제공.
함수형 프로그래밍 패턴과의 일관성.

# 3.next_func. 자동 상속 메커니즘
torch.autograd.Func. 상속 시 next_functions attribute의 자동 포함.
User가 명시적으로 next_functions 구현 불필요.

PyTorch의 내부 process는

*   Operation 결과에 해당 operation type의 grad_fn 연결.
*   grad_fn의 next_functions에 input tensor들의 grad_fn 자동 저장.


# 3-1.next_func. attri. 구조



*   Tuple of tuples 형태의 type임
*   각 tuple의 첫 번째 요소는 input tensor의 grad_fn, 두 번째 요소는 input index.
*   Input tensor들이 forward 함수에 전달된 순서와 동일한 순서로 저장됨.
*   requires_grad=False인 tensor의 경우 (None, 0) 형태로 저장.




# 3-2.다중 output인 op.예시

In [ ]:
# 하나의 텐서를 두 부분으로 분할
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
a, b = torch.chunk(x, 2)  # 두 개의 결과물 생성
c = a.sum() + b.sum()

print("c.grad_fn:", c.grad_fn)
print("c.grad_fn.next_functions:", c.grad_fn.next_functions)

# 각 SumBackward0의 next_functions
for i, (grad_fn, idx) in enumerate(c.grad_fn.next_functions):
    print(f"grad_fn[{i}].next_functions:", grad_fn.next_functions)

c.grad_fn: <AddBackward0 object at 0x7bb307ef3fd0>
c.grad_fn.next_functions: ((<SumBackward0 object at 0x7bb307ef20e0>, 0), (<SumBackward0 object at 0x7bb307ef0070>, 0))
grad_fn[0].next_functions: ((<SplitBackward0 object at 0x7bb307ef30d0>, 0),)
grad_fn[1].next_functions: ((<SplitBackward0 object at 0x7bb307ef30d0>, 1),)




*   torch.chunk는 하나의 tensor를 여러 부분으로 나누는 op, torch.chunk(tensor, chunks, dim=0): 지정된 dimension을 따라 tensor를 chunks 개수만큼 균등하게 분할.
*   각 chunk는 원본 tensor의 크기를 chunks로 나눈 크기를 가짐.
*   dim=0을 기준으로 분할하나, dim parameter를 통해 다른 dimension 기준 분할 가능.
*   각 chunk에 대한 gradient 계산 시 ChunkBackward의 서로 다른 output(index 0과 1)




In [ ]:
class MyCustomFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, t):
      ctx.save_for_backward(x, y)
      return x * y

    @staticmethod
    def backward(ctx, grad_output):
      x, y = ctx.saved_tensors
      return grad_output * y, grad_output * x

custom_op = MyCustomFunction.apply
a = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(2.0, requires_grad=True)
c = custom_op(a, b)

print("c.grad_fn:", c.grad_fn)
print("c.grad_fn.next_functions:", c.grad_fn.next_functions)

c.grad_fn: <torch.autograd.function.MyCustomFunctionBackward object at 0x7bb402b9dbf0>
c.grad_fn.next_functions: ((<AccumulateGrad object at 0x7bb307d552a0>, 0), (<AccumulateGrad object at 0x7bb307d55e40>, 0))




*   backpropagation 시 전체 computation graph의 추적 가능.
*   Autograd system의 효율적인 gradient computation의 기반.



https://dabletech.oopy.io/99645b3b-d58a-4225-9253-3589b98c5bb1
라는 링크 안에 pytorch autograd(오태호 작성자)
내용 추천해주심

15,17 정도는 해보면 좋겟지만, 20까지 하면 직접 만들 수 있음!